# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassThrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

#### Runnable 예제

#### RunnableLambda 예제

#### RunnablePassThrough 예제

#### RunnableSequence 예제

#### RunnableParallel 예제

#### LCEL Chain 예제

## Chain과 Chain간의 연결

# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```